In [1]:
from joblib import load

# 모델 불러오기
loaded_model = load("../Data/model.joblib")

print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [2]:
import pandas as pd
import holidays
from datetime import datetime

# 특징 생성 함수
def create_features_for_datetime(dt):
    """datetime에서 특징을 생성"""
    return {
        'datetime': dt,
        '연': dt.year,
        '월': dt.month,
        '일': dt.day,
        '시간': dt.hour,
        '요일': dt.weekday(),
        '주말': 1 if dt.weekday() >= 5 else 0,
        '공휴일': 1 if dt in holidays.KR() else 0,
    }

# 미래 예측 함수
def predict_passenger_count(model, future_datetime):
    """미래 승객 수 예측"""
    feature = create_features_for_datetime(future_datetime)
    feature_df = pd.DataFrame([feature])
    feature_columns = ['연', '월', '일', '시간', '요일', '주말', '공휴일']
    feature_input = feature_df[feature_columns]
    prediction = model.predict(feature_input)
    prediction = [max(0, round(float(val))) for val in prediction[0]]
    return {
        '10대': prediction[0],
        '20대': prediction[1],
        '30대': prediction[2],
        '40대': prediction[3],
        '50대 이상': prediction[4],
        '우대권': prediction[5],
        '외국인': prediction[6]
    }

# 특정 날짜의 시간대별 예측 함수
def predict_specific_date_all_hours(model, date_input):
    """특정 날짜의 0시부터 23시까지 모든 시간대의 승객 수 예측"""
    specific_date = datetime.strptime(date_input, '%Y%m%d')
    predictions = []
    for hour in range(24):
        if 1 <= hour <= 4:
            continue
        current_time = specific_date.replace(hour=hour, minute=0, second=0, microsecond=0)
        predicted_count = predict_passenger_count(model, current_time)
        predictions.append({
            'datetime': current_time.strftime('%Y-%m-%d %H:%M:%S'),
            **predicted_count
        })
    return pd.DataFrame(predictions)

In [13]:
# 특정 날짜 예측 실행
date_input = "20241223"  # YYYYMMDD 형식으로 날짜 입력
predictions_for_date = predict_specific_date_all_hours(loaded_model, date_input)

# 승객수 합계 열 추가
predictions_for_date['승객수 합계'] = predictions_for_date[['10대', '20대', '30대', '40대', '50대 이상', '우대권', '외국인']].sum(axis=1)

# 결과 출력
predictions_for_date

,datetime,10대,20대,30대,40대,50대 이상,우대권,외국인,승객수 합계
0,2024-12-23 00:00:00,3,0,0,15,2,1,0,21
1,2024-12-23 05:00:00,29,70,149,86,48,269,0,651
2,2024-12-23 06:00:00,187,455,910,534,289,590,0,2965
3,2024-12-23 07:00:00,205,1258,3682,2229,351,449,0,8174
4,2024-12-23 08:00:00,391,2476,7245,4443,686,543,4,15788
5,2024-12-23 09:00:00,368,2271,6822,4132,630,785,7,15015
6,2024-12-23 10:00:00,568,1278,2674,1599,863,894,14,7890
7,2024-12-23 11:00:00,427,988,2021,1202,664,957,19,6278
8,2024-12-23 12:00:00,439,1000,2064,1212,666,882,24,6287
9,2024-12-23 13:00:00,515,1161,2398,1394,813,991,28,7300
